# Camada refined
Transformações para obtenção de zona de características para treinamento do modelo

* Padronização em lower case
* Remover palavras comuns e conectores
* Remover espaço em branco e pontuações
* Atribuição de id hash para garantir a rastreabilidade dos textos

### Configs

In [2]:
pip install gensim

In [3]:
!pip install spacy
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 29.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import pandas as pd
from gensim.utils import simple_preprocess

### Leitura da camada Trusted

In [5]:
df_claims = pd.read_parquet('../dataset/trusted/claims.parquet.snappy')
df_claims

,company_name,date,uf,city,customer_claim,company_response
0,Correios,2021-04-13,MG,São João Del Rei,"Boa tarde ! No dia 08/04/2021, por volta das 1...","Prezado Cliente, para que possamos verificar o..."
1,Samsung,2021-04-25,MG,Belo Horizonte,Fui realizar essa ultima atualização do androi...,"Caro cliente, Conforme contato realizado atrav..."
2,Vivo - Telefônica,2021-04-14,MG,Caratinga,"Ao contatar a empresa, solicitei a 2ª via de c...",Prezada Sra. Vitoria Realizamos tentativa de c...
3,Vivo - Telefônica,2021-04-13,MG,Carmópolis de Minas,Fiz uma reclamação anteriormente a respeito de...,Prezada Sra. Juliana Realizamos tentativa de c...
4,Vivo - Telefônica,2021-04-14,MG,Santa Bárbara do Leste,Solicitei a Vivo plano para o número (33) XXXX...,Contatamos o responsável pela manifestação no ...
...,...,...,...,...,...,...
378569,Claro Celular,2014-06-27,MG,Belo Horizonte,Em 16/05/2012 - contrataçao dos serviços de te...,"Sr. Vinicius, em atenção à reclamação sobre o ..."
378570,Walmart.com,2014-06-27,MG,Formiga,Na madrugada do último dia 29/05/14 efetuei um...,O Walmart.com informa que a compra do cliente ...
378571,Claro Celular,2014-06-27,MG,Juiz de Fora,eu liguei para operadora pedindo para efetuar ...,"Sra. Caroline, em atenção à reclamação sobre o..."
378572,Claro Celular,2014-06-27,MG,Belo Horizonte,"A CLARO, não está prestando o serviços devidos...","Sra. Rejane, em atenção à reclamação sobre cob..."


In [6]:
df_claims.count()

,0
company_name,378574
date,378574
uf,378574
city,378574
customer_claim,378574
company_response,375107


In [7]:
df_claims.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378574 entries, 0 to 378573
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   company_name      378574 non-null  object        
 1   date              378574 non-null  datetime64[ns]
 2   uf                378574 non-null  object        
 3   city              378574 non-null  object        
 4   customer_claim    378574 non-null  object        
 5   company_response  375107 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 20.2+ MB


### Criação Id Hash
Criação de ID hash para garantir rastreabilidade dos registros

In [8]:
import hashlib
def create_hash(row):
    # Concatena todos os valores das colunas e gera o hash sha1
    concatenated_string = ''.join(row.astype(str))
    return hashlib.sha1(concatenated_string.encode('utf-8')).hexdigest()

# Adiciona coluna 'id'
df_claims['id'] = df_claims.apply(create_hash, axis=1)
df_claims

,company_name,date,uf,city,customer_claim,company_response,id
0,Correios,2021-04-13,MG,São João Del Rei,"Boa tarde ! No dia 08/04/2021, por volta das 1...","Prezado Cliente, para que possamos verificar o...",4b286594b1bfaa6c60a7f729fa0746f48c612c5d
1,Samsung,2021-04-25,MG,Belo Horizonte,Fui realizar essa ultima atualização do androi...,"Caro cliente, Conforme contato realizado atrav...",ba6785160fd5e43eeed575fbcdbd2abf05935cfc
2,Vivo - Telefônica,2021-04-14,MG,Caratinga,"Ao contatar a empresa, solicitei a 2ª via de c...",Prezada Sra. Vitoria Realizamos tentativa de c...,f607b8cecea9c11da27a6fc45f9c1bc43c1f375e
3,Vivo - Telefônica,2021-04-13,MG,Carmópolis de Minas,Fiz uma reclamação anteriormente a respeito de...,Prezada Sra. Juliana Realizamos tentativa de c...,04a82557dbde804b3ecf6fd357b91346c972e398
4,Vivo - Telefônica,2021-04-14,MG,Santa Bárbara do Leste,Solicitei a Vivo plano para o número (33) XXXX...,Contatamos o responsável pela manifestação no ...,a9ab2f2c4b24d64d90c2f5d00e0a85a4f5c187a4
...,...,...,...,...,...,...,...
378569,Claro Celular,2014-06-27,MG,Belo Horizonte,Em 16/05/2012 - contrataçao dos serviços de te...,"Sr. Vinicius, em atenção à reclamação sobre o ...",8cc3e46d81667f3906668ac04e8ebe0b70099c88
378570,Walmart.com,2014-06-27,MG,Formiga,Na madrugada do último dia 29/05/14 efetuei um...,O Walmart.com informa que a compra do cliente ...,e35b701e491bb5fa96f3ab9be6c8561002344fd9
378571,Claro Celular,2014-06-27,MG,Juiz de Fora,eu liguei para operadora pedindo para efetuar ...,"Sra. Caroline, em atenção à reclamação sobre o...",77b002e25da07990cc3d2e8876d282730ac681dc
378572,Claro Celular,2014-06-27,MG,Belo Horizonte,"A CLARO, não está prestando o serviços devidos...","Sra. Rejane, em atenção à reclamação sobre cob...",bf91d708e308e5d77f801862cca7ad487b6d4b19


## Tranformações
Utilizando o pacote *simple_preprocess*, responsável por gerar uma lista de tokens

Tranformações aplicadas:
*   Padronização em lower case
*   Desconsideração de palavras com 1 caracter e mais de 15 caracteres
*   Desconsideração de espaços em branco, números e caracteres especiais


Doc: https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [ ]:
df_claims['tokens_customer_claim'] = df_claims['customer_claim'].apply(lambda x: simple_preprocess(x, deacc=True))
df_claims

### Removendo stop words

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_sm')

# conjunto de stop words a partir do modelo SpaCy
stop_words = set(nlp.Defaults.stop_words)

In [ ]:
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

In [ ]:
df_claims['tokens_customer_claim_clean'] = df_claims['tokens_customer_claim'].apply(remove_stopwords)
df_claims

,company_name,date,uf,city,customer_claim,company_response,id,tokens_customer_claim,tokens_customer_claim_clean
0,Correios,2021-04-13,MG,São João Del Rei,"Boa tarde ! No dia 08/04/2021, por volta das 1...","Prezado Cliente, para que possamos verificar o...",4b286594b1bfaa6c60a7f729fa0746f48c612c5d,"[boa, tarde, no, dia, por, volta, das, fui, ag...","[dia, volta, agencia, correios, localizada, ba..."
1,Samsung,2021-04-25,MG,Belo Horizonte,Fui realizar essa ultima atualização do androi...,"Caro cliente, Conforme contato realizado atrav...",ba6785160fd5e43eeed575fbcdbd2abf05935cfc,"[fui, realizar, essa, ultima, atualizacao, do,...","[realizar, ultima, atualizacao, android, apare..."
2,Vivo - Telefônica,2021-04-14,MG,Caratinga,"Ao contatar a empresa, solicitei a 2ª via de c...",Prezada Sra. Vitoria Realizamos tentativa de c...,f607b8cecea9c11da27a6fc45f9c1bc43c1f375e,"[ao, contatar, empresa, solicitei, via, de, co...","[contatar, empresa, solicitei, via, contas, de..."
3,Vivo - Telefônica,2021-04-13,MG,Carmópolis de Minas,Fiz uma reclamação anteriormente a respeito de...,Prezada Sra. Juliana Realizamos tentativa de c...,04a82557dbde804b3ecf6fd357b91346c972e398,"[fiz, uma, reclamacao, anteriormente, respeito...","[fiz, reclamacao, anteriormente, respeito, deb..."
4,Vivo - Telefônica,2021-04-14,MG,Santa Bárbara do Leste,Solicitei a Vivo plano para o número (33) XXXX...,Contatamos o responsável pela manifestação no ...,a9ab2f2c4b24d64d90c2f5d00e0a85a4f5c187a4,"[solicitei, vivo, plano, para, numero, xxxxx, ...","[solicitei, vivo, plano, numero, xxxxx, aconte..."
...,...,...,...,...,...,...,...,...,...
378569,Claro Celular,2014-06-27,MG,Belo Horizonte,Em 16/05/2012 - contrataçao dos serviços de te...,"Sr. Vinicius, em atenção à reclamação sobre o ...",8cc3e46d81667f3906668ac04e8ebe0b70099c88,"[em, contratacao, dos, servicos, de, telefonia...","[contratacao, servicos, telefonia, movel, cont..."
378570,Walmart.com,2014-06-27,MG,Formiga,Na madrugada do último dia 29/05/14 efetuei um...,O Walmart.com informa que a compra do cliente ...,e35b701e491bb5fa96f3ab9be6c8561002344fd9,"[na, madrugada, do, ultimo, dia, efetuei, uma,...","[madrugada, ultimo, dia, efetuei, compra, site..."
378571,Claro Celular,2014-06-27,MG,Juiz de Fora,eu liguei para operadora pedindo para efetuar ...,"Sra. Caroline, em atenção à reclamação sobre o...",77b002e25da07990cc3d2e8876d282730ac681dc,"[eu, liguei, para, operadora, pedindo, para, e...","[liguei, operadora, pedindo, efetuar, cancelam..."
378572,Claro Celular,2014-06-27,MG,Belo Horizonte,"A CLARO, não está prestando o serviços devidos...","Sra. Rejane, em atenção à reclamação sobre cob...",bf91d708e308e5d77f801862cca7ad487b6d4b19,"[claro, nao, esta, prestando, servicos, devido...","[claro, nao, prestando, servicos, devidos, dia..."


In [ ]:
df_claims.count()

,0
company_name,378574
date,378574
uf,378574
city,378574
customer_claim,378574
company_response,375107
id,378574
tokens_customer_claim,378574
tokens_customer_claim_clean,378574


## Salvamento em parquet

In [ ]:
# Sem stop_words para análises futuras
rfzd_claims_clean = (df_claims[['tokens_customer_claim_clean', 'id']]
               .rename(columns={'tokens_customer_claim_clean': 'tokens_customer_claim'}))

rfzd_claims_clean.to_parquet('../dataset/refined/rfzd_claims_clean.parquet', engine='pyarrow', compression='snappy')

In [ ]:
rfzd_claims = (df_claims[['tokens_customer_claim', 'id']])

rfzd_claims .to_parquet('../dataset/refined/rfzd_claims.parquet', engine='pyarrow', compression='snappy')